In [1]:
# default_exp core

# dproc

> API details.

In [2]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [3]:
#export
import json
import os
import glob
import io
import re
import zipfile
from datetime import date
from datetime import datetime
from functools import wraps
from pathlib import Path
import logging
import numpy as np
import pandas as pd
import pandas_profiling
import pandas_flavor as pf
import janitor
from dataclasses import dataclass
from typing import List, Dict, Tuple, Sequence

In [4]:
#export
@dataclass
class Meta:
    _datatypes: pd.DataFrame = None
    _definition: pd.DataFrame = None
    _description: pd.DataFrame = None
    _entity_definition: pd.DataFrame = None
    _entity: str = None
    _is_unique_id: List = None
    _is_mandatory: List = None
    _name_clean: List = None
    _name_raw: List = None
    _name_dashboard: List = None
    _today = datetime.today().strftime("%Y-%m-%d")
    _profile_report: pandas_profiling.__init__.ProfileReport = None
        
    
    @property
    def profile_report(self) -> pandas_profiling.__init__.ProfileReport:
        return self._profile_report
    
    @profile_report.setter
    def profile_report(self, profile_report) -> None:
        self._profile_report = profile_report
        
    @property
    def definition(self) -> pd.DataFrame:
        self._update_definition()
        return self._definition

    @definition.setter
    def definition(self, df: pd.DataFrame) -> None:
        self._definition = df
        
    @property
    def entity(self) -> str:
        return self._entity

    @entity.setter
    def entity(self, value: str) -> None:
        self._entity = value
        
    def _update_definition(self):
        self._definition['entity'] = self._entity
        self._definition['name_raw'] = self._name_raw
        self._definition['name_clean'] = list(self._name_clean.values())
        self._definition['name_dashboard'] = list(self._name_dashboard.values())
        
        if self._is_unique_id == {}:
            self._definition['is_unique_id'] = False
        else:
            self._definition['is_unique_id'] = list(self._is_unique_id.values())
        
        if self._is_mandatory == {}:
            self._definition['is_mandatory'] = True
        else:
            self._definition['is_mandatory'] = list(self._is_mandatory.values())
            
        if self._datatypes == {}:
            self._definition['datatype'] = None
        else:
            self._definition['datatype'] = list(self._datatypes.values())
            
        if self._description == {}:
            self._definition['description'] = None
        else:
            self._definition['description'] = list(self._description.values())
            
    def _update_meta(self):
        self._name_raw = list(self._definition['name_raw'].values)
        self._name_clean = dict(zip(self._name_raw, list(self._definition['name_clean'].values)))
        self._name_dashboard = dict(zip(list(self._definition['name_clean'].values),
                                        list(self._definition['name_dashboard'].values)))
        self._is_unique_id = dict(zip(list(self._definition['name_clean'].values), 
                                      list(self._definition['is_unique_id'].values)))
        self._is_mandatory = dict(zip(list(self._definition['name_clean'].values), 
                                      list(self._definition['is_mandatory'].values)))
        self._datatypes = dict(zip(list(self._definition['name_clean'].values),
                                   list(self._definition['datatype'].values)))
        self._description = dict(zip(list(self._definition['name_clean'].values),
                                     list(self._definition['description'].values)))
            
    @property
    def entity_definition(self) -> pd.DataFrame:
        return self._entity_definition
    
    @property
    def today(self):
        return self._today
    
    @property
    def today(self):
        return self._today
        
    @property
    def name_clean(self):
        return self._name_clean

    @name_clean.setter
    def name_clean(self, names: Dict) -> Dict:
        self._name_clean = names
        
    @property
    def name_raw(self):
        return self._name_raw

    @name_raw.setter
    def name_raw(self, names: List) -> None:
        self._name_raw = list(names)
    
    @property
    def name_dashboard(self):
        return self._name_dashboard

    @name_dashboard.setter
    def name_dashboard(self, names: Dict) -> None:
        self._name_dashboard = list(name)
        
    @property
    def is_unique_id(self):
        return self._is_unique_id

    @is_unique_id.setter
    def is_unique_id(self, cols: Dict) -> None:
        self._is_unique_id = cols
    
    @property
    def is_mandatory(self):
        return self._is_mandatory

    @is_mandatory.setter
    def is_mandatory(self, cols: Dict) -> None:
        self._is_mandatory = cols
    
    @property
    def description(self):
        return self._description
    
    @description.setter
    def description(self, description: Dict) -> None:
        self.description = description
        
    @property
    def datatypes(self):
        return self._datatypes
    
    @datatypes.setter
    def datatypes(self, datatypes: Dict) -> None:
        self._datatypes = datatypes
        self._definition = self._definition['datatype'] = self._datatypes
    
    def make_definition(self, df=None, name_raw=None, name_clean=None, name_dashboard=None, 
                        is_unique_id=None, is_mandatory=None, datatypes=None, description=None):
        """Make definition out of properties"""
        dd = pd.DataFrame()
        
        try:
            dd['entity'] = meta.entity        
        except:
            print('No entity name. Please provide an enity name by either providing an entity argument or setting meta.entity.')
            return None
        
        if (self.name_raw is None) and (df is None):
            print('No raw names to start with. Please provide a list of raw names or a dataframe.')
            return None
        try:
            if name_raw:
                dd['name_raw'] = name_raw
            elif list(df.columns):
                name_raw = list(df.columns)
                dd['name_raw'] = name_raw
            elif self._name_raw:
                dd['name_raw'] = self.name_raw
            else:
                dd['name_raw'] = None
        except:
            print('No raw names to start with. Please provide a list of raw names or a dataframe.')
            return None
        
        if name_clean:
            dd['name_clean'] = name_clean
        else:
            name_clean = list(pd.DataFrame(columns=name_raw).clean_names().columns)
            dd['name_clean'] = name_clean
        
        if name_dashboard:
            dd['name_dashboard'] = name_dashboard
        else:
            name_dashboard = []
            for name in name_clean:
                ''.join([x.capitalize() for x in name.split('_')]) 
                name_dashboard.append(''.join([x.capitalize() for x in name.split('_')])) 
            dd['name_dashboard'] = name_dashboard
                
        if is_unique_id:
            dd['is_unique_id'] = is_unique_id
        else:
            dd['is_unique_id'] = False
            
        if is_mandatory:
            dd['is_mandatory'] = is_mandatory
        else:
            dd['is_mandatory'] = True
        
        if datatypes:
            dd['datatype'] = datatypes
        else:  
            dd['datatype'] = pd.DataFrame(df.infer_objects().dtypes).reset_index()[0].astype(str).values
        
        if description:
            dd['description'] = description
        elif self._description:
            dd['description'] = self._description
        else: 
            dd['description'] = None
                
        self.definition = dd
        self._update_meta()

In [5]:
#hide


In [6]:
#export
meta = Meta()

**Example: make_definition**

As a starting point name the entity.

In [7]:
meta.entity = 'meteor-landing'

In [8]:
meteor_landing = pd.read_csv('examples/meteorite-landings.csv')

In [9]:
meta.make_definition(meteor_landing)

In [10]:
meta.definition

,entity,name_raw,name_clean,name_dashboard,is_unique_id,is_mandatory,datatype,description
0,meteor-landing,Name,name,Name,False,True,object,None
1,meteor-landing,ID,id,Id,False,True,int64,None
2,meteor-landing,Name Type,name_type,NameType,False,True,object,None
3,meteor-landing,Rec-class,rec_class,RecClass,False,True,object,None
4,meteor-landing,MASs,mass,Mass,False,True,float64,None
5,meteor-landing,Fall,fall,Fall,False,True,object,None
6,meteor-landing,YEAR,year,Year,False,True,float64,None
7,meteor-landing,Rec_lat,rec_lat,RecLat,False,True,float64,None
8,meteor-landing,Rec_long,rec_long,RecLong,False,True,float64,None
9,meteor-landing,Geo Location,geo_location,GeoLocation,False,True,object,None


The result is a definition which can be used as a start providing reasonable defaults.
You can now save this file and edit in an external program or further tweak. 

In [11]:
meta.description

{'name': None,
 'id': None,
 'name_type': None,
 'rec_class': None,
 'mass': None,
 'fall': None,
 'year': None,
 'rec_lat': None,
 'rec_long': None,
 'geo_location': None}

In [12]:
meta.name_dashboard

{'name': 'Name',
 'id': 'Id',
 'name_type': 'NameType',
 'rec_class': 'RecClass',
 'mass': 'Mass',
 'fall': 'Fall',
 'year': 'Year',
 'rec_lat': 'RecLat',
 'rec_long': 'RecLong',
 'geo_location': 'GeoLocation'}

In [13]:
meta.is_mandatory

{'name': True,
 'id': True,
 'name_type': True,
 'rec_class': True,
 'mass': True,
 'fall': True,
 'year': True,
 'rec_lat': True,
 'rec_long': True,
 'geo_location': True}

Setting the unique id:

In [14]:
meta.is_unique_id['id'] = True

Adding a description for the attribute **name**:

In [15]:
meta.description['name'] = 'the name of the meteorite (typically a location, often modified with a number, year, composition, etc)'

In [16]:
meta.definition

,entity,name_raw,name_clean,name_dashboard,is_unique_id,is_mandatory,datatype,description
0,meteor-landing,Name,name,Name,False,True,object,the name of the meteorite (typically a locatio...
1,meteor-landing,ID,id,Id,True,True,int64,None
2,meteor-landing,Name Type,name_type,NameType,False,True,object,None
3,meteor-landing,Rec-class,rec_class,RecClass,False,True,object,None
4,meteor-landing,MASs,mass,Mass,False,True,float64,None
5,meteor-landing,Fall,fall,Fall,False,True,object,None
6,meteor-landing,YEAR,year,Year,False,True,float64,None
7,meteor-landing,Rec_lat,rec_lat,RecLat,False,True,float64,None
8,meteor-landing,Rec_long,rec_long,RecLong,False,True,float64,None
9,meteor-landing,Geo Location,geo_location,GeoLocation,False,True,object,None


In [17]:
#export
@pf.register_dataframe_method
def step_rename_cols(df, mapping=None):
    """Rename columns (raw column names -> clean column names)"""
    if not mapping:
        mapping = dict(
            zip(
                meta.definition["name_raw"].values,
                meta.definition["name_clean"].values,
            )
        )
    df = df.rename(columns=mapping)
    return df

**Example: step_rename_cols**

In [18]:
#### Examp

In [19]:
#export
@pf.register_dataframe_method
def step_replace_missing_with_nan(df, missing_values=None):
    """Replace all missing values with nan"""
    if missing_values is None:
        missing_values = ["nan", "NAN", "NaN", "none", "None", "NONE", '""', "''", ""]
    for col in df.select_dtypes(include="object").columns:
        df.loc[df[col].isin(missing_values), col] = np.nan
    return df

In [20]:
#export
@pf.register_dataframe_method
def step_remove_not_needed_cols(df, cols=None):
    """Remove not needed columns"""
    if not cols:
        cols = list(meta.definition[meta.definition.is_mandatory == False]["name_clean"].values)    
    df = df.drop(columns=cols)
    return df

In [21]:
#export
@pf.register_dataframe_method
def step_remove_rows_with_missing_ids(df, id_cols=None):
    """Remove rows with invalid ids"""
    if not id_cols:
        id_cols = list(meta.definition[meta.definition.is_unique_id == True]["name_clean"].values)        
    df = df.dropna(subset=id_cols)
    return df

In [22]:
#export
@pf.register_dataframe_method
def step_remove_duplicate_rows(df, id_cols=None, keep="last"):
    """Remove duplicate rows"""
    if not id_cols:
        id_cols = meta.definition[meta.definition.is_unique_id == True]["name_clean"].values
    df.drop_duplicates(subset=id_cols, keep=keep, inplace=True)
    return df

In [23]:
#export
@pf.register_dataframe_method
def step_format_dates(df, cols=None, date_format=None):
    """Format date columns"""
    if not cols:
        cols = meta.definition[meta.definition.datatype.isin(["date", "datetime"])]["name_clean"].values
    for col in cols:
        if date_format:
            df[col] = pd.to_datetime(df[col], format=date_format)
        else:
            df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
    return df

In [24]:
#export
def _round_float(value, decimal_places=2):
    '''Helper function to round s if s is float'''
    m = re.match("(\d+\.\d+)",value.__str__())
    try:
        r = round(float(m.groups(0)[0]),decimal_places)
    except:
        r = value
    return r

In [25]:
#export
@pf.register_dataframe_method
def step_format_round_numeric_cols(df, cols=None, decimal_places=2):
    """Round columns to defined number of decimal places"""
    if not cols:
        cols = meta.definition[meta.definition["dtype"].isin(["float"])]["name_clean"].values
    for col in cols:    
        df[col] = df[col].apply(_round_float, args=(decimal_places,))
    return df

In [26]:
#export
@pf.register_dataframe_method
def step_change_dtypes(df, date_format="%Y-%m-%d %H:%M:%S"):
    """Change datatypes"""
    _entity_definition = meta.definition[meta.definition.is_mandatory == True]
    cols = list(
        zip(
            _entity_definition["name_clean"].values,
            _entity_definition["dtype"].values,
        )
    )
    for col in cols:
        if col[1] == "int":
            df[col[0]] = pd.to_numeric(df[col[0]], errors='coerce').astype('Int64')
        elif col[1] == "float":
            df[col[0]] = df[col[0]].astype(float)
        elif col[1] == "str":
            df[col[0]] = df[col[0]].astype(str)
        elif col[1] == "category":
            df[col[0]] = df[col[0]].astype("category")
        elif col[1] == "bool":
            df[col[0]] = df[col[0]].astype(bool)
    return df

In [27]:
@pf.register_dataframe_method
def make_profile(df, title=None, file_path=None):
    """Make profile report"""
    meta.profile = ProfileReport(df,
                        title=title,
                        html={'style':{'full_width':True}})
    return df

In [28]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
